# Gauss-Newton for Logistic Regression

In [42]:
include("../jl/UCBAdmit_Logit.jl")
include("../jl/admissionll.jl")
include("../jl/admissiongr.jl")

Variable `Y` has the observed variable.
Variable `X` has the explanatory variables.


admitgr (generic function with 1 method)

## Add Backtracking to Gradient Descent

In [43]:
function backtrack(S,∇S₀,p,X₀,Y, X, counts; α₀ = 1.0, ρ = 0.5, c = 0.2)
  S₀ = S(X₀, Y, X, counts)
  relaxSlope = c*p'*(∇S₀'*S₀)
  condition(step) = 0.5*dot(S₀,S₀) + step*relaxSlope[1]

    α = deepcopy(α₀)
  Sₐ = S₀ #Ensures appropriate scope of Sₐ
  try
    Sₐ = S(X₀ + α*p, Y, X, counts)
  catch
  end
  funcEvals = 2
  while 0.5*dot(Sₐ,Sₐ) > condition(α)
    α = ρ*α
    try
      Sₐ = S(X₀ + α*p, Y, X, counts)
    catch
      continue
    end
    funcEvals += 1
  end

  return α, funcEvals
end

backtrack (generic function with 4 methods)

In [44]:
function logitback(S,∇S, B; 
        Y=Y, X=X[:,1:7], counts=X[:,8], ϵ=1e-8, maxiter = 25)
  i = 0
  S₀ = S(B, Y, X, counts)
  ∇S₀ = ∇S(B, Y, X, counts)
  while norm(∇S₀) > ϵ && i <= maxiter
    i += 1
    #Search Direction
    ∇S₀ = ∇S(B, Y, X, counts)
    p = ∇S₀'
    #Step Length
    α, evals = backtrack(S,∇S₀,p,B,Y,X,counts)
    #Update Parameter
    B += α*p
    #Update Gradient
    S₀ = S(B, Y, X, counts)
  end

  return B, i
end

logitback (generic function with 2 methods)

In [57]:
B = [1 1 1 1 1 1 1]
sol, iters = logitback(admitll, admitgr, B; 
    Y=Y, X=X[:,1:7], counts=X[:,8], ϵ=1e-8, maxiter=10000);
sol'

7×1 Adjoint{Float64,Array{Float64,2}}:
  0.5820508211476487 
  0.09986975221729864
 -0.04339722231887582
 -1.2625971299553376 
 -1.2946056224343268 
 -1.7393047193207791 
 -3.3064784485822507 

In [58]:
iters

10001